## In this kernel we convert LEVEL5 Lyft data (NuScenes format) to KITTI format, which is usually used in public repositories. After this you can search for repos, that solve KITTI 3d-detection task.

!pip install -qqq -U git+https://github.com/stalkermustang/nuscenes-devkit.git

In [18]:
from pathlib import Path
from PIL import Image

In [19]:
# dir with all input data from Kaggle
INP_DIR = Path('/Users/kanhua/Downloads/3d-object-detection-for-autonomous-vehicles')

In [20]:
# dir with index json tables (scenes, categories, logs, etc...)
TABLES_DIR = INP_DIR.joinpath('train_data')

In [21]:
# Adjust the dataroot parameter below to point to your local dataset path.
# The correct dataset path contains at least the following four folders (or similar): images, lidar, maps
!ln -s {INP_DIR}/train_images images
!ln -s {INP_DIR}/train_maps maps
!ln -s {INP_DIR}/train_lidar lidar

ln: images/train_images: File exists
ln: maps/train_maps: File exists
ln: lidar/train_lidar: File exists


In [22]:
DATA_DIR = Path().absolute() 
# Empty init equals '.'.
# We use this because we link train dirs to current dir (cell above)

In [23]:
# dir to write KITTY-style dataset
STORE_DIR = DATA_DIR.joinpath('kitti_format')

In [24]:
!python -m lyft_dataset_sdk.utils.export_kitti nuscenes_gt_to_kitti -h

INFO: Showing help with the command 'export_kitti.py nuscenes_gt_to_kitti -- --help'.

NAME
    export_kitti.py nuscenes_gt_to_kitti - Converts nuScenes GT fromatted annotations to KITTI format.

SYNOPSIS
    export_kitti.py nuscenes_gt_to_kitti LYFT_DATAROOT TABLE_FOLDER <flags>

DESCRIPTION
    Converts nuScenes GT fromatted annotations to KITTI format.

POSITIONAL ARGUMENTS
    LYFT_DATAROOT
        Where lyft dataset stored (root dir).
    TABLE_FOLDER
        folder with tables (json files).

FLAGS
    --lidar_name=LIDAR_NAME
        Name of the lidar sensor. Only one lidar allowed at this moment.
    --get_all_detections=GET_ALL_DETECTIONS
        If True, will write all bboxes in PointCloud and use only FrontCamera.
    --parallel_n_jobs=PARALLEL_N_JOBS
        Number of threads to parralel processing.
    --samples_count=SAMPLES_COUNT
        Number of samples to convert.

NOTES
    You can also use flags syntax for POSITIONAL ARGUMENTS


In [25]:
# convertation to KITTY-format
!python -m lyft_dataset_sdk.utils.export_kitti nuscenes_gt_to_kitti \
        --lyft_dataroot {DATA_DIR} \
        --table_folder {TABLES_DIR} \
        --samples_count 2000 \
        --parallel_n_jobs 2 \
        --get_all_detections True \
        --store_dir {STORE_DIR}

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 10.8 seconds.
Reverse indexing ...
Done reverse indexing in 3.9 seconds.
100%|███████████████████████████████████████| 2000/2000 [11:33<00:00,  2.88it/s]


In [26]:
# check created (converted) files. velodyne = LiDAR poinclouds data (in binary)
!ls {STORE_DIR}/velodyne | head -2

0032f6f11828b3925466a7699fbe598938a652560260c87f4742aa164b1d5021.bin
006385e4f70c8f0c98ea85dbc7b89c054f9ae7f019332f975716cf8d45f20a03.bin


In [27]:
# render converted data for check. Currently don't support multithreading :(
!python -m lyft_dataset_sdk.utils.export_kitti render_kitti \
        --store_dir {STORE_DIR}

Rendering 3d boxes projected from 3d KITTI format
  2%|▉                                      | 46/2000 [03:20<2:52:09,  5.29s/it]Traceback (most recent call last):
  File "/Users/kanhua/miniconda3/envs/convert-kitti/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kanhua/miniconda3/envs/convert-kitti/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kanhua/miniconda3/envs/convert-kitti/lib/python3.7/site-packages/lyft_dataset_sdk/utils/export_kitti.py", line 316, in <module>
    fire.Fire(KittiConverter)
  File "/Users/kanhua/miniconda3/envs/convert-kitti/lib/python3.7/site-packages/fire/core.py", line 138, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/Users/kanhua/miniconda3/envs/convert-kitti/lib/python3.7/site-packages/fire/core.py", line 471, in _Fire
    target=component.__name__)
  File "/Users/kanhua/miniconda3/envs/convert-kitti/lib/python3

In [28]:
# Script above write images to 'render' folder
# in store_dir (where we have converted dataset)
RENDER_DIR = STORE_DIR.joinpath('render')

In [29]:
# get all rendered files
all_renders = list(RENDER_DIR.glob('*'))
all_renders.sort()

In [30]:
# render radar data (bird view) and camera data with bboxes

In [31]:
Image.open(all_renders[0])

IndexError: list index out of range

In [ ]:
Image.open(all_renders[1])

## I'm use rendering only for check success converting. 

## Can be used to visualize NN predictions for test lyft set (visual metric estimation :D)

!rm -rf {STORE_DIR}